In [1]:
from collections import Counter
from enum import IntEnum
from pathlib import Path
import shutil
import subprocess
from tqdm import tqdm
from typing import Dict, List, Optional

Use examples in [python cookbook, 3rd Edition](https://www.amazon.com/Python-Cookbook-Third-David-Beazley/dp/1449340377) as test cases to verify the correctness of the implementation.

In [2]:
IWASM = Path(shutil.which("iwasm"))

PY2WASM = Path(shutil.which("py2wasm"))

PYTHON_COOKBOOK = Path.cwd().joinpath("./cookbook/src")
assert PYTHON_COOKBOOK.exists(), "use `git submodule update` to fetch cookbook"


In [3]:
class CaseErrorCode(IntEnum):
    OK = 0
    COMPILATION_FAILURE = 1
    WASM_EXECUTION_FAILURE = 2
    PY_EXECUTION_FAILURE = 3
    DIFFERENT_RESULT = 4
    BYPASS = 5

    def __str__(self):
        return f'{self.name}'

class CaseResult:
    def __init__(self, error_code: CaseErrorCode, msg: str):
        self.error_code = error_code
        self.msg = msg

    def __repr__(self):
        return f"{self.error_code}\n{self.msg}"


For each case, we are going to:

1. compile the python script to a WebAssembly core module 
2. run the python script 
3. run the WebAssembly core module
4. compare both execution results of step 2 and step 3

In [4]:
def compile_py_2_wasm(py2wasm_bin: Path, py_file: Path, out_dir: Optional[Path]) -> Path:
    if not out_dir is None:
        out_dir.mkdir(parents=True, exist_ok=True)
        wasm_file = out_dir.joinpath(py_file.stem + ".wasm")
    else:
        wasm_file = py_file.with_suffix(".wasm")
    
    cmd = f"{py2wasm_bin} {py_file} -o {wasm_file}"
    subprocess.run(cmd, cwd=Path.cwd(), capture_output=True, check=True, shell=True, universal_newlines=True, text=True)
    return wasm_file

def execute_py(py_file: Path) -> subprocess.CompletedProcess:
    """
    Since relative pathes are used in examples.
    """
    cwd = py_file.parent
    cmd = f"python {py_file}"
    return subprocess.run(cmd, cwd=cwd, capture_output=True, check=True, shell=True, universal_newlines=True, text=True)

def execute_wasm(iwasm_bin: Path, wasm_file: Path) -> subprocess.CompletedProcess:
    """
    To avoid the WASI pre-opens problem, run under the same directory with the wasm file
    """
    cwd = wasm_file.parent
    cmd = f"{iwasm_bin} --dir=. {wasm_file.relative_to(cwd)}"
    return subprocess.run(cmd, cwd=cwd, capture_output=True, check=True, shell=True, universal_newlines=True, text=True)
    

def compare_result(expected: str, actual: str) -> bool:
    return expected == actual

##################################################
def execute_case(py_file: Path, out_dir: Optional[Path]) -> CaseResult:
    try:
        wasm_file = compile_py_2_wasm(PY2WASM, py_file, out_dir)
    except subprocess.CalledProcessError:
        return CaseResult(CaseErrorCode.COMPILATION_FAILURE, "")

    try:
        p = execute_wasm(IWASM, wasm_file)
    except subprocess.CalledProcessError as e:
        return CaseResult(CaseErrorCode.WASM_EXECUTION_FAILURE, e.stderr if e.stderr else e.stdout)
    output_from_wasm = p.stdout

    try:
        p = execute_py(py_file)
    except subprocess.CalledProcessError as e:
        return CaseResult(CaseErrorCode.PY_EXECUTION_FAILURE, e.stderr if e.stderr else e.stdout)
    output_from_py = p.stdout

    if not compare_result(output_from_py, output_from_wasm):
        return CaseResult(CaseErrorCode.DIFFERENT_RESULT, f"{'>' * 40}\n{output_from_py}{'-' * 20} V.S. {'-' * 20}\n{output_from_wasm}{'<' * 40}")
    
    return CaseResult(CaseErrorCode.OK, "")

def visit_case(py_file: Path, out_dir: Optional[Path]) -> CaseResult:
    print(f"py_file={py_file}, out_dir={out_dir}")
    return CaseResult(CaseErrorCode.OK, "")

def execute_test(chapter: Path, bypass: List[str]) -> Dict[str, CaseResult]:
    # list all sub-directories in chapter
    chapter_tests = [x for x in chapter.iterdir() if x.is_dir()]

    # walk
    ret = {}
    for test_dir in tqdm(chapter_tests):
        if test_dir.name in bypass:
            ret[case_name] = CaseResult(CaseErrorCode.BYPASS, "")
            continue

        # assume all .py under the test_dir are test cases
        py_files = [x for x in test_dir.iterdir() if x.suffix == ".py"]

        for py_file in py_files:
            result = execute_case(py_file, None)
            # result = visit_case(py_file, None)
            case_name = f"{test_dir.name}.{py_file.stem}"
            ret[case_name] = result
    
    return ret

def summarize_result(results: Dict[str, CaseResult]) -> None:
    counter = Counter([x.error_code for x in results.values()])
    print()
    print(counter)
    print()

    # print failures
    for k,v in results.items():
        if v.error_code != CaseErrorCode.OK:
            print(f"{k} FAILED:\n{v}")
            print()

## Chapter 1. Data Structures And Algorithms

`DIFFERENT_RESULT` in *finding_out_what_two_dictionaries_have_in_common* isn't a bug 😄. It is caused by the different order of the keys in the dictionary.

`EXECUTION_FAILURE` in *transforming_and_reducing_data_at_the_same_time* needs more investigation 🔎. My first guess is about preopen.

In [5]:
chapter_1 = PYTHON_COOKBOOK.joinpath("1")
chapter_1_result = execute_test(chapter_1, [])
summarize_result(chapter_1_result)

100%|██████████| 16/16 [04:38<00:00, 17.43s/it]


Counter({<CaseErrorCode.OK: 0>: 16, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 1})

transforming_and_reducing_data_at_the_same_time.example FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example.py", line 6, in <module>
    files = os.listdir(os.path.expanduser('~'))
    
FileNotFoundError: [Errno 44] No such file or directory: '~'




### Investigation of *transforming_and_reducing_data_at_the_same_time*

In [6]:
! iwasm --dir=. --dir=/home/vscode ./cookbook/src/1/transforming_and_reducing_data_at_the_same_time/example.wasm

Traceback (most recent call last):
  File "./example.py", line 6, in <module>
    files = os.listdir(os.path.expanduser('~'))
    
FileNotFoundError: [Errno 44] No such file or directory: '~'


In [7]:
! iwasm --dir=. --dir=~ ./cookbook/src/1/transforming_and_reducing_data_at_the_same_time/example.wasm

error while pre-opening directory ~: 2



## Chapter 2. Strings And Text

There are `try...except...` in some cases. Need to deep dive into the wasm implementation.

In [8]:
chapter_2 = PYTHON_COOKBOOK.joinpath("2")
chapter_2_result = execute_test(chapter_2, [])
summarize_result(chapter_2_result)

100%|██████████| 13/13 [04:36<00:00, 21.25s/it]


Counter({<CaseErrorCode.OK: 0>: 13, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 2})

writing_a_simple_recursive_descent_parser.plyexample FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


matching_strings_using_shell_wildcard_patterns.example FAILED:
WASM_EXECUTION_FAILURE
Fatal Python error: init_sys_streams: can't initialize sys standard streams
Python runtime state: core initialized
ImportError: Frozen object named 'io' is invalid

Current thread 0x00000000 (most recent call first):
  <no Python frame>




## Chapter 3. Numbers, Dates, And Times

In [9]:
chapter_3 = PYTHON_COOKBOOK.joinpath("3")
chapter_3_result = execute_test(chapter_3, [])
summarize_result(chapter_3_result)

100%|██████████| 2/2 [00:36<00:00, 18.05s/it]


Counter({<CaseErrorCode.OK: 0>: 2})



## Chapter 4. Iterators And Generators

In [10]:
chapter_4 = PYTHON_COOKBOOK.joinpath("4")
chapter_4_result = execute_test(chapter_4, [])
summarize_result(chapter_4_result)

100%|██████████| 10/10 [02:52<00:00, 17.23s/it]


Counter({<CaseErrorCode.OK: 0>: 9, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 2})

creating_data_processing_pipelines.example FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example.py", line 3, in <module>
    import gzip
    
  File "./gzip.py", line 9, in <module>
ImportError: dynamic libraries are not implemented in wasi


how_to_flatten_a_nested_sequence.example FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example.py", line 3, in <module>
    from collections import Iterable
    
ImportError: cannot import name 'Iterable' from 'collections' (./collections/__init__.py)




## Chapter 5. Files And I/O

In [11]:
chapter_5 = PYTHON_COOKBOOK.joinpath("5")
chapter_5_result = execute_test(chapter_5, [])
summarize_result(chapter_5_result)

100%|██████████| 6/6 [01:41<00:00, 16.97s/it]


Counter({<CaseErrorCode.OK: 0>: 4, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 2})

wrapping_an_existing_file_descriptor_as_a_file_object.echo FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./echo.py", line 26, in <module>
    echo_server(('', 25000))

  File "./echo.py", line 17, in echo_server
    sock = socket(AF_INET, SOCK_STREAM)

  File "./socket.py", line 232, in __init__
OSError: [Errno 58] Not supported


adding_or_changing_the_encoding_of_an_already_open_file.example FAILED:
WASM_EXECUTION_FAILURE
Exception: out of bounds memory access




## Chapter 6. Data Encoding And Processing

In [12]:
chapter_6 = PYTHON_COOKBOOK.joinpath("6")
chapter_6_result = execute_test(chapter_6, [])
summarize_result(chapter_6_result)

100%|██████████| 8/8 [03:50<00:00, 28.80s/it]


Counter({<CaseErrorCode.OK: 0>: 7, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 5, <CaseErrorCode.DIFFERENT_RESULT: 4>: 2})

reading_and_writing_json_data.example FAILED:
DIFFERENT_RESULT
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])
ACME
50
490.1
{"__classname__": "Point", "x": 3, "y": 4}
3
4
-------------------- V.S. --------------------
OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])
ACME
50
490.1
{"__classname__": "Point", "x": 3, "y": 4}
3
4
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

parsing_xml_documents_with_namespaces.example FAILED:
DIFFERENT_RESULT
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<Element '{http://www.w3.org/1999/xhtml}html' at 0x7f34b8318360>
Hello World
-------------------- V.S. --------------------
<Element '{http://www.w3.org/1999/xhtml}html' at 0xc77338>
Hello World
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

reading_nested_and_variable_sized_binary_structures.example4 FAILED:
WASM_EXECUT

## Chapter 7. Functions

In [13]:
chapter_7 = PYTHON_COOKBOOK.joinpath("7")
chapter_7_result = execute_test(chapter_7, [])
summarize_result(chapter_7_result)

100%|██████████| 7/7 [03:23<00:00, 29.05s/it]


Counter({<CaseErrorCode.OK: 0>: 6, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 5})

inlining_callback_functions.example FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


making_an_n-argument_callable_work_as_a_callable_with_fewer_arguments.example2 FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example2.py", line 21, in <module>
    p = Pool()

  File "./multiprocessing/context.py", line 118, in Pool
  File "./multiprocessing/pool.py", line 30, in <module>
  File "./multiprocessing/connection.py", line 22, in <module>
ImportError: dynamic libraries are not implemented in wasi


making_an_n-argument_callable_work_as_a_callable_with_fewer_arguments.example3 FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example3.py", line 16, in <module>
    serv = TCPServer(('', 15000), partial(EchoHandler, ack=b'RECEIVED:'))

  File "./socketserver.py", line 452, in __init__
  File "./socket.py", line 232, in __init__
OSErr

## Chapter 8. Classes And Objects

In [14]:
chapter_8 = PYTHON_COOKBOOK.joinpath("8")
chapter_8_result = execute_test(chapter_8, [])
summarize_result(chapter_8_result)

100%|██████████| 24/24 [13:43<00:00, 34.29s/it]


Counter({<CaseErrorCode.OK: 0>: 32, <CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 16, <CaseErrorCode.DIFFERENT_RESULT: 4>: 1})

managing_memory_in_cyclic_data_structures.example FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


making_objects_support_the_context_manager_protocol.example2 FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example2.py", line 24, in <module>
    with conn as s:

  File "./example2.py", line 11, in __enter__
    sock = socket(self.family, self.type)

  File "./socket.py", line 232, in __init__
OSError: [Errno 58] Not supported


making_objects_support_the_context_manager_protocol.example1 FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example1.py", line 26, in <module>
    with c as s:

  File "./example1.py", line 13, in __enter__
    self.sock = socket(self.family, self.type)

  File "./socket.py", line 232, in __init__
OSError: [Errno 58] Not supported


implementing_the_visitor_p

## Chapter 9. Metaprogramming


In [15]:
chapter_9 = PYTHON_COOKBOOK.joinpath("9")
chapter_9_result = execute_test(chapter_9, [])
summarize_result(chapter_9_result)

100%|██████████| 23/23 [10:24<00:00, 27.14s/it]


Counter({<CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 26, <CaseErrorCode.OK: 0>: 7, <CaseErrorCode.DIFFERENT_RESULT: 4>: 3})

using_metaclasses_to_control_instance_creation.example1 FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


preserving_function_metadata_when_writing_decorators.example FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


defining_a_decorator_that_takes_an_optional_argument.example FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


defining_decorators_as_classes.example2 FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


defining_decorators_as_classes.example1 FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


defining_decorators_as_classes.example3 FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


defining_a_decorator_that_takes_arguments.example FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


disassembling_python_by

## Chapter 10. Modules And Packages

In [16]:
chapter_10 = PYTHON_COOKBOOK.joinpath("10")
chapter_10_result = execute_test(chapter_10, [])
summarize_result(chapter_10_result)

100%|██████████| 4/4 [02:46<00:00, 41.71s/it]


Counter({<CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 5, <CaseErrorCode.OK: 0>: 4})

monkeypatching_modules_on_import.example2 FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


loading_modules_from_a_remote_machine_using_import_hooks.metaexample FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./metaexample.py", line 10, in <module>
    import urlimport
    
  File "./urlimport.py", line 6, in <module urlimport>
    from urllib.request import urlopen
    
  File "./urllib/request.py", line 88, in <module>
  File "./http/client.py", line 71, in <module>
  File "./email/parser.py", line 12, in <module>
  File "./email/feedparser.py", line 27, in <module>
  File "./email/_policybase.py", line 7, in <module>
  File "./email/header.py", line 17, in <module>
ModuleNotFoundError: No module named 'email.base64mime'


loading_modules_from_a_remote_machine_using_import_hooks.pathexample FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last)

## Chapter 11. Network And Web Programming

In [17]:
chapter_11 = PYTHON_COOKBOOK.joinpath("11")
chapter_11_result = execute_test(chapter_11, [])
summarize_result(chapter_11_result)

100%|██████████| 12/12 [23:44<00:00, 118.69s/it]


Counter({<CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 46, <CaseErrorCode.OK: 0>: 4})

event_driven_io_explained.tcpclient FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./tcpclient.py", line 3, in <module>
    s = socket(AF_INET, SOCK_STREAM)
    
  File "./socket.py", line 232, in __init__
OSError: [Errno 58] Not supported


event_driven_io_explained.threadpool FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./threadpool.py", line 63, in <module>
    pool = ThreadPoolHandler(16)

  File "./threadpool.py", line 7, in __init__
    self.signal_done_sock, self.done_sock = socket.socketpair()

  File "./socket.py", line 631, in socketpair
  File "./socket.py", line 232, in __init__
OSError: [Errno 58] Not supported


event_driven_io_explained.udpclient FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./udpclient.py", line 2, in <module>
    s = socket(AF_INET, SOCK_DGRAM)
    
  File "./socket.py", line 232, in __

## Chapter 12. Concurrency

In [18]:
chapter_12 = PYTHON_COOKBOOK.joinpath("12")
chapter_12_result = execute_test(chapter_12, [])
summarize_result(chapter_12_result)

100%|██████████| 13/13 [08:39<00:00, 39.97s/it]


Counter({<CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 25, <CaseErrorCode.OK: 0>: 3})

how_to_start_and_stop_threads.example FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


defining_an_actor_task.actor FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


defining_an_actor_task.tagged FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


defining_an_actor_task.worker FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


using_generators_as_an_alternative_to_threads.netsched FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./netsched.py", line 159, in <module>
    sched.run()

  File "./netsched.py", line 62, in run
    r = task.send(msg)

  File "./netsched.py", line 137, in server_loop
    s = Socket(socket(AF_INET,SOCK_STREAM))

  File "./socket.py", line 232, in __init__
OSError: [Errno 58] Not supported


storing_thread_specific_state.example2 FAILED:
WASM_EXECUTION_FAILURE
Exceptio

## Chatper 13. Utility Scripting And System Administration

In [19]:
chapter_13 = PYTHON_COOKBOOK.joinpath("13")
chapter_13_result = execute_test(chapter_13, [])
summarize_result(chapter_13_result)

100%|██████████| 11/11 [03:52<00:00, 21.10s/it]


Counter({<CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 9, <CaseErrorCode.OK: 0>: 4})

parsing_command_line_options.search FAILED:
WASM_EXECUTION_FAILURE
usage: search.wasm [-h] -p pattern [-v] [-o OUTFILE] [--speed {fast,slow}]
                   [filename ...]
search.wasm: error: the following arguments are required: -p/--pat


prompting_for_a_password_at_runtime.example FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example.py", line 3, in <module>
    user = getpass.getuser()
    
  File "./getpass.py", line 168, in getuser
ModuleNotFoundError: No module named 'pwd'


generating_a_range_of_ip_addresses_from_a_cidr_address.example FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example.py", line 25, in <module>
    for a in cidr_range('12:3456:78:90ab:cd:ef01:23:34/125'):
 
  File "./example.py", line 9, in cidr_range
    addr_bytes = inet_pton(family, address)

OSError: can't use AF_INET6, IPv6 is disabled


putting_limits_on_me

## Chapter 14. Testing, Debugging, And Exceptions

In [20]:
chapter_14 = PYTHON_COOKBOOK.joinpath("14")
chapter_14_result = execute_test(chapter_14, [])
summarize_result(chapter_14_result)

100%|██████████| 7/7 [11:02<00:00, 94.63s/it] 


Counter({<CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 5, <CaseErrorCode.OK: 0>: 3})

logging_test_output_to_a_file.test FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


make_your_programs_run_faster.example FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


skipping_or_anticipating_test_failures.test FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


testing_for_exceptional_conditions_in_unit_tests.test FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch


profiling_and_timing_your_program.timethis FAILED:
WASM_EXECUTION_FAILURE
Exception: indirect call type mismatch




## Chapter 15. C Extensions

In [21]:
chapter_15 = PYTHON_COOKBOOK.joinpath("15")
chapter_15_result = execute_test(chapter_15, [])
summarize_result(chapter_15_result)

100%|██████████| 16/16 [15:27<00:00, 57.98s/it] 


Counter({<CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 31})

working_with_c_strings_of_dubious_encoding.setup FAILED:
WASM_EXECUTION_FAILURE
./setup.py:2: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.core import setup, Extension
usage: setup.wasm [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
   or: setup.wasm --help [cmd1 cmd2 ...]
   or: setup.wasm --help-commands
   or: setup.wasm cmd --help

error: no commands supplied


working_with_c_strings_of_dubious_encoding.example FAILED:
WASM_EXECUTION_FAILURE
Traceback (most recent call last):
  File "./example.py", line 1, in <module>
    import sample
    
ModuleNotFoundError: No module named 'sample'


passing_null_terminated_strings_to_c_libraries.setup FAILED:
WASM_EXECUTION_FAILURE
./setup.py:2: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools o

## Summary

In [26]:
chapter_x_results = {
    **chapter_1_result,
    **chapter_2_result,
    **chapter_3_result,
    **chapter_4_result,
    **chapter_5_result,
    **chapter_6_result,
    **chapter_7_result,
    **chapter_8_result,
    **chapter_9_result,
    **chapter_10_result,
    **chapter_11_result,
    **chapter_12_result,
    **chapter_13_result,
    **chapter_14_result,
    **chapter_15_result,
}


x_counters = Counter([x.error_code for x in chapter_x_results.values()])
print()
print(x_counters)
print()



Counter({<CaseErrorCode.WASM_EXECUTION_FAILURE: 2>: 180, <CaseErrorCode.OK: 0>: 114, <CaseErrorCode.DIFFERENT_RESULT: 4>: 6})

